## Read Data and Calculate Allocation

In [ ]:
import pandas as pd
import numpy as np
from plotly.io import write_html
import plotly.graph_objects as go
from plotly.subplots import make_subplots


def initialize_data():
    # Cumulative supply data
    supply_df = pd.DataFrame({
        'Week': ['Jan Wk1', 'Jan Wk2', 'Jan Wk3', 'Jan Wk4', 'Jan Wk5'],
        'Total Supply': [200, 230, 270, 320,
                         380]  # Wk1 represents initial inventory
    }).set_index('Week')

    # Initial builds (count against Wk1 supply)
    built_df = pd.DataFrame({
        'Product': ['Superman', 'Superman Mini', 'Superman Plus'],
        'Jan Wk1': [70, 60, 70]
    }).set_index('Product')

    # Cumulative demand forecasts
    demand_df = pd.DataFrame({
        'Superman': {
            'Jan Wk2': 85,
            'Jan Wk3': 100,
            'Jan Wk4': 110,
            'Jan Wk5': 120
        },
        'Superman Mini': {
            'Jan Wk2': 40,
            'Jan Wk3': 60,
            'Jan Wk4': 70,
            'Jan Wk5': 75
        }
    })

    # Superman Plus CUMULATIVE channel demand
    sp_demand_df = pd.DataFrame({
        'Online Store': {
            'Jan Wk1': 17,
            'Jan Wk2': 20,
            'Jan Wk3': 30,
            'Jan Wk4': 40,
            'Jan Wk5': 50
        },
        'Retail Store': {
            'Jan Wk1': 12,
            'Jan Wk2': 15,
            'Jan Wk3': 25,
            'Jan Wk4': 30,
            'Jan Wk5': 35
        },
        'Reseller AMR': {
            'Jan Wk1': 16,
            'Jan Wk2': 20,
            'Jan Wk3': 25,
            'Jan Wk4': 30,
            'Jan Wk5': 35
        },
        'Reseller Europe': {
            'Jan Wk1': 4,
            'Jan Wk2': 5,
            'Jan Wk3': 10,
            'Jan Wk4': 15,
            'Jan Wk5': 15
        },
        'Reseller PAC': {
            'Jan Wk1': 21,
            'Jan Wk2': 25,
            'Jan Wk3': 30,
            'Jan Wk4': 35,
            'Jan Wk5': 40
        }
    })

    return supply_df, built_df, demand_df, sp_demand_df


def calculate_allocations(supply_df, built_df, demand_df, sp_demand_df):
    alloc_dfs = {
        'Superman': pd.DataFrame(columns=['Allocation']),
        'Superman Mini': pd.DataFrame(columns=['Allocation']),
        'Superman Plus': pd.DataFrame(columns=sp_demand_df.columns)
    }

    # Track cumulative allocations
    cumulative = {
        'Superman': built_df.loc['Superman', 'Jan Wk1'],
        'Superman Mini': built_df.loc['Superman Mini', 'Jan Wk1'],
        'Superman Plus': built_df.loc['Superman Plus', 'Jan Wk1'],
        'Total': built_df['Jan Wk1'].sum()
    }

    for week in ['Jan Wk2', 'Jan Wk3', 'Jan Wk4', 'Jan Wk5']:
        # Calculate remaining supply
        remaining_supply = supply_df.loc[week,
                                         'Total Supply'] - cumulative['Total']

        # Allocate to priority products (incremental)
        superman_needed = max(
            0, demand_df.loc[week, 'Superman'] - cumulative['Superman'])
        superman_mini_needed = max(
            0,
            demand_df.loc[week, 'Superman Mini'] - cumulative['Superman Mini'])
        total_priority_needed = superman_needed + superman_mini_needed

        priority_allocation = min(total_priority_needed, remaining_supply)

        if total_priority_needed > 0:
            superman_alloc = priority_allocation * (superman_needed /
                                                    total_priority_needed)
            superman_mini_alloc = priority_allocation * (superman_mini_needed /
                                                         total_priority_needed)
        else:
            superman_alloc = 0
            superman_mini_alloc = 0

        alloc_dfs['Superman'].loc[week, 'Allocation'] = superman_alloc
        alloc_dfs['Superman Mini'].loc[week,
                                       'Allocation'] = superman_mini_alloc

        # Update cumulative allocations
        cumulative['Superman'] += superman_alloc
        cumulative['Superman Mini'] += superman_mini_alloc
        cumulative['Total'] += priority_allocation

        # Remaining for Superman Plus
        remaining = remaining_supply - priority_allocation

        # Calculate incremental demand for Superman Plus channels
        prev_week = f"Jan Wk{int(week[-1])-1}"
        sp_incremental = (sp_demand_df.loc[week] -
                          sp_demand_df.loc[prev_week]).to_dict()

        # Special handling for Jan Wk4 PAC Reseller
        if week != 'Jan Wk5':
            pac_incremental = sp_incremental['Reseller PAC']
            pac_alloc = min(pac_incremental, remaining)
            alloc_dfs['Superman Plus'].loc[week, 'Reseller PAC'] = pac_alloc
            remaining -= pac_alloc

            # Allocate remaining to other channels
            other_channels = [c for c in sp_incremental if c != 'Reseller PAC']
            total_other_demand = sum(sp_incremental[c] for c in other_channels)

            if total_other_demand > 0:
                for channel in other_channels:
                    alloc = (sp_incremental[channel] /
                             total_other_demand) * remaining
                    alloc_dfs['Superman Plus'].loc[week, channel] = alloc
        else:
            # Normal proportional allocation
            total_demand = sum(sp_incremental.values())
            if total_demand > 0:
                for channel in sp_incremental:
                    alloc = (sp_incremental[channel] /
                             total_demand) * remaining
                    alloc_dfs['Superman Plus'].loc[week, channel] = alloc

        # Update cumulative Superman Plus allocation
        sp_alloc = alloc_dfs['Superman Plus'].loc[week].sum()
        cumulative['Superman Plus'] += sp_alloc
        cumulative['Total'] += sp_alloc

    return alloc_dfs, cumulative


# Initialize and calculate
supply_df, built_df, demand_df, sp_demand_df = initialize_data()
alloc_dfs, final_allocation = calculate_allocations(supply_df, built_df,
                                                    demand_df, sp_demand_df)
alloc_dfs['Superman Plus'] = alloc_dfs['Superman Plus'].astype('float').round()

# Show allocation
print("Full Superman Plus allocations:")
print(alloc_dfs['Superman Plus'])

## Draw Plots

In [ ]:
def create_waterfall_chart(alloc_dfs, demand_df, weeks):
    # Prepare data
    initial_instock_superman = 70
    initial_instock_superman_mini = 60
    # Prepare data including Wk1 initial builds
    superman_data = {
        'cum_demand': [0] + demand_df['Superman'].values.tolist(),
        'cum_alloc': [initial_instock_superman] +
        alloc_dfs['Superman'].values.ravel().tolist()[:-1] +
        [(alloc_dfs['Superman'].sum() + initial_instock_superman).values[0]]
    }

    superman_mini_data = {
        'cum_demand': [0] + demand_df['Superman Mini'].values.tolist(),
        'cum_alloc': [initial_instock_superman_mini] +
        alloc_dfs['Superman Mini'].values.ravel().tolist()[:-1] +
        [(alloc_dfs['Superman Mini'].sum() +
          initial_instock_superman_mini).values[0]]
    }

    # Create combined figure
    fig = make_subplots(rows=1,
                        cols=2,
                        subplot_titles=("Superman", "Superman Mini"),
                        shared_yaxes=True)

    # Add waterfall bars
    for i, (product, data) in enumerate(
            zip(['Superman', 'Superman Mini'],
                [superman_data, superman_mini_data])):
        # Calculate incremental values for waterfall
        fig.add_trace(go.Waterfall(
            name=f"{product} Allocation",
            x=weeks,
            y=data['cum_alloc'],
            base=0,
            measure=['absolute'] + ["relative"] * (len(weeks) - 2) +
            ['absolute'],
            increasing={"marker": {
                "color": "#636EFA"
            }},
            customdata=[0] + alloc_dfs[product].values.ravel().tolist(),
            hovertemplate=("Week: %{x}<br>"
                           "Allocation: %{customdata:.0f}<br>"
                           "Cum Alloc: %{y:.0f}<br>"
                           "<extra></extra>")),
                      row=1,
                      col=i + 1)

        # Add demand line
        fig.add_trace(go.Scatter(x=weeks[1:],
                                 y=data['cum_demand'][1:],
                                 name=f"{product} Demand",
                                 line=dict(color='orange', width=2,
                                           dash='dot'),
                                 hovertemplate=("Week: %{x}<br>"
                                                "Demand: %{y:.0f}<br>"
                                                "<extra></extra>")),
                      row=1,
                      col=i + 1)

    # Update layout
    fig.update_layout(title="Superman/Superman Mini Allocation vs Demand",
                      waterfallgap=0.2,
                      height=500,
                      showlegend=True,
                      legend=dict(orientation="v", yanchor="bottom", y=0.8),
                      xaxis_title="Week",
                      yaxis_title="Cumulative Allocation",
                      xaxis2_title="Week",
                      plot_bgcolor="rgba(240,240,240,0.5)",
                      hovermode='x unified')

    # Format y-axis to show full cumulative scale
    max_demand = max(max(superman_data['cum_demand']),
                     max(superman_mini_data['cum_demand']))
    fig.update_yaxes(range=[0, max_demand * 1.1])

    return fig


def create_superman_plus_stacked_waterfall(alloc_dfs,
                                           sp_demand_df,
                                           initial_total=70):
    weeks = ['Jan Wk1', 'Jan Wk2', 'Jan Wk3', 'Jan Wk4', 'Jan Wk5']
    channels = [
        'Online Store', 'Retail Store', 'Reseller AMR', 'Reseller Europe',
        'Reseller PAC'
    ]
    colors = ['#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A']

    # Calculate cumulative allocations
    cumulative = [initial_total]
    for week in weeks[1:]:
        weekly_total = sum(alloc_dfs['Superman Plus'].loc[week, channels])
        cumulative.append(cumulative[-1] + weekly_total)

    # Create figure with proper trace structure
    fig = go.Figure()

    # 1. Initial build (single bar)
    fig.add_trace(
        go.Bar(
            x=['Jan Wk1'],
            y=[initial_total],
            name='Initial Build',
            marker_color='lightgray',
            marker_pattern_shape="/",
            width=[0.5],  # Narrower bar for Wk1
            hovertemplate="Initial Build: 70<extra></extra>"))

    # 2. Weekly increments (stacked by channel)
    base = np.array([float(initial_total)] +
                    (alloc_dfs['Superman Plus'].sum(1).cumsum() +
                     initial_total).values.astype('float').tolist()[:-1])
    for idx, channel in enumerate(channels):
        y_values = np.array([
            alloc_dfs['Superman Plus'].loc[week, channel] for week in weeks[1:]
        ])
        fig.add_trace(
            go.Bar(
                x=weeks[1:],
                y=y_values,
                name=channel,
                marker_color=colors[idx],
                base=base,  # Start from initial build
                offsetgroup='increments',
                customdata=y_values,
                hovertemplate=(f"Channel: {channel}<br>"
                               "Week: %{x}<br>"
                               "Incremental: %{customdata:.0f}<br>"
                               "Cumulative: %{y:.0f}<br>"
                               "<extra></extra>")))
        base += y_values

    # 3. Demand line (skip Wk1)
    total_demand = [
        sum(sp_demand_df.loc[week, channels]) for week in weeks[1:]
    ]
    fig.add_trace(
        go.Scatter(x=weeks[1:],
                   y=total_demand,
                   name="Superman Plus Demand",
                   line=dict(color='black', width=3, dash='dot'),
                   hovertemplate="Week: %{x}<br>Demand: %{y:.0f}"
                   "<extra></extra>"))

    # Update layout
    fig.update_layout(title="Superman Plus Allocation (Stacked)",
                      barmode='stack',
                      height=600,
                      showlegend=True,
                      legend=dict(orientation="h", yanchor="bottom", y=1.02),
                      xaxis_title="Week",
                      yaxis_title="Cumulative Units",
                      hovermode="x unified",
                      plot_bgcolor="rgba(240,240,240,0.5)",
                      bargap=0.3,
                      xaxis={
                          'categoryorder': 'array',
                          'categoryarray': weeks
                      })

    # Format y-axis
    max_value = max(max(total_demand), cumulative[-1])
    fig.update_yaxes(range=[0, max_value * 1.1])

    return fig


def create_allocation_table_fig(alloc_dfs, weeks):
    # Prepare data
    data = []
    columns = [
        'Week', 'Superman', 'Superman Mini', 'Online Store', 'Retail Store',
        'Reseller AMR', 'Reseller Europe', 'Reseller PAC', 'Total Allocated'
    ]

    for week in weeks:
        row = [week]
        row.append(alloc_dfs['Superman'].loc[week, 'Allocation'])
        row.append(alloc_dfs['Superman Mini'].loc[week, 'Allocation'])
        row.append(alloc_dfs['Superman Plus'].loc[week, 'Online Store'])
        row.append(alloc_dfs['Superman Plus'].loc[week, 'Retail Store'])
        row.append(alloc_dfs['Superman Plus'].loc[week, 'Reseller AMR'])
        row.append(alloc_dfs['Superman Plus'].loc[week, 'Reseller Europe'])
        row.append(alloc_dfs['Superman Plus'].loc[week, 'Reseller PAC'])
        row.append(sum(row[1:]))  # Total

        # Format numbers to 1 decimal place
        row = [f"{x:.0f}" if isinstance(x, (int, float)) else x for x in row]
        data.append(row)

    # Create table figure
    fig = go.Figure(data=[
        go.Table(
            header=dict(
                values=columns,
                align="center",
                font=dict(size=13, color="white"),
                fill_color="#4E79A7",  # header
                line_color="darkslategray"),
            cells=dict(
                values=list(zip(*data)),
                align="center",
                font=dict(size=12),
                fill_color=["white"] + ["#F7F7F7"] * 7 + ['white'],  # banded
                line_color="lightgray"))
    ])

    fig.update_layout(title='Allocation Summary by Week',
                      height=250,
                      margin=dict(l=10, r=10, b=10, t=50))

    return fig


def create_allocation_visualizations(alloc_dfs, supply_df, demand_df,
                                     sp_demand_df):
    weeks = supply_df.index[1:]  # Use supply_df index as canonical week order

    # 1. Supply vs Demand Overview
    fig_supply_demand = go.Figure()

    # Add supply line (using supply_df index)
    fig_supply_demand.add_trace(
        go.Scatter(x=supply_df.index,
                   y=supply_df['Total Supply'],
                   name='Cumulative Supply',
                   hovertemplate="%{x}<br>%{y}",
                   line=dict(color='green', width=4)))

    # Add demand lines (aligned to same weeks)
    total_demand = pd.concat([demand_df, sp_demand_df], axis=1).sum(1).values
    priority_demand = demand_df.sum(1).values

    fig_supply_demand.add_trace(
        go.Scatter(x=supply_df.index,
                   y=[np.nan] + total_demand.tolist(),
                   name='Total Demand',
                   hovertemplate="%{x}<br>%{y}",
                   line=dict(color='red', dash='dash')))

    fig_supply_demand.add_trace(
        go.Scatter(x=supply_df.index,
                   y=[np.nan] + priority_demand.tolist(),
                   name='Superman/Superman Mini Demand',
                   hovertemplate="%{x}<br>%{y}",
                   line=dict(color='orange', dash='dash')))

    fig_supply_demand.add_trace(
        go.Bar(x=supply_df.index,
               y=(supply_df.sum(1) -
                  (demand_df.sum(1) + sp_demand_df.sum(1))).values.tolist(),
               name='Supply - Demand',
               hovertemplate="%{x}<br>%{y}",
               marker_color='grey'))

    fig_supply_demand.update_layout(
        title='Material A: Cumulative Supply vs Demand',
        xaxis_title='Week',
        yaxis_title='Quantity',
        hovermode='x unified',
        plot_bgcolor="rgba(240,240,240,0.5)",
        legend=dict(orientation="h", yanchor="bottom", y=1),
        height=500)

    # 2. Priority Products Allocation
    fig_priority = create_waterfall_chart(alloc_dfs, demand_df, ['Jan Wk1'] +
                                          demand_df.index.tolist())

    # 3. Superman Plus Channel Allocation
    fig_sp_channels = create_superman_plus_stacked_waterfall(
        alloc_dfs, sp_demand_df)

    # 4. Allocation Summary Table
    fig_table = create_allocation_table_fig(alloc_dfs, weeks)

    return fig_supply_demand, fig_priority, fig_sp_channels, fig_table


# Generate visualizations with aligned x-axes
fig1, fig2, fig3, fig_table = create_allocation_visualizations(
    alloc_dfs, supply_df, demand_df, sp_demand_df)

In [ ]:
def save_centered_dashboard(figures, filename="allocation_dashboard.html"):
    with open(filename, 'w') as f:
        # Write HTML header with CSS
        f.write("""
        <!DOCTYPE html>
        <html>
        <head>
            <style>
                .plot-container {
                    width: 90%;
                    margin: 0 auto;
                    padding: 20px;
                }
                body {
                    font-family: Arial, sans-serif;
                }
            </style>
        </head>
        <body>
        """)

        # Write each figure wrapped in container
        for i, fig in enumerate(figures):
            f.write('<div class="plot-container">')
            fig.write_html(f,
                           full_html=False,
                           include_plotlyjs='cdn' if i == 0 else False,
                           config={'responsive': True})
            f.write('</div>')

        # Close HTML
        f.write("""
        </body>
        </html>
        """)

    print(f"Dashboard saved to {filename}")


all_figures = [fig1, fig2, fig3, fig_table]
save_centered_dashboard(all_figures)

# Display figures
fig1.show()
fig2.show()
fig3.show()
fig_table.show()